# Thesis: Comparison of two search results using keyword co-occurrence visualization
## Purpose
This notebook takes exported results from two searches and outputs a nodes file and an edges file for use with Gephi. The intent is to visualize how two search terms, in this case, 'social justice' and 'social injustice' do or don't overlap and may have terms that are unique to each search. Such information or visualization may be of benefit to those trying to understand and explore new fields.

## Method
This notebook cleans and prepares data downloaded from Web of Science for 2010-2020 on two independent searches: <br>
TOPIC = 'social justice' and, 
TOPIC = 'social 'injustice'
restricted to articles only
<br>
Currently, the process is:<br>
 - download data from WoS of search results restricted to years defined (2010-2020)
 - import the WoS data into this notebook to clean the keywords and perform counts and percentages to find mutually shared terms and observe the percentage of time one keyword appears in the search of the other list of keywords. 
- Exports a truncated node file of the most 1000 frequent terms in .csv format.
- using the truncated node file, creates an edges list of term associations by document
- Exports an edges .csv file.
- The two files may be imported into Gephi into a common workspace to visualize. 

### current issues:
- [ ] add bibliographic coupling to the output for nodes and edges 
- [x] add Leiden algorithm for clustering
- [ ] compare abstract topic clusters with cosign similarity, LDA, and Ward Heirarchical?


### long-term goals
- [ ] fix pub year - currently pub year ('PY') is not included. Might be nice for longitudinal investigations
- [ ] would like to use Networkx for more control. (and subsequently Streamlit for embedding and Heroku for deployment)
- [ ] or integrate with Gephi Streaming for more real-time?





# step 1: import data
This is for the .txt files of Full Records from Web of Science. 

In [1]:
# import libraries
import pandas as pd
import os, glob
import re
import string # for removing punctuation

#set to see all columns
pd.set_option('max_columns', None)

In [2]:
# Define relative path to folder containing the text files

files_SJ_folder = "/Users/nicolapoppy/OneDrive - Dalhousie University/Thesis/Data and stuff/wos/SJ"


files_SI_folder = "/Users/nicolapoppy/OneDrive - Dalhousie University/Thesis/Data and stuff/wos/SI"


# Create dataframe list by using a list comprehension
files1 = []
files1 = [pd.read_csv(file, delimiter='\t') for file in glob.glob(os.path.join(files_SJ_folder ,"*.txt"))]

# create list of all files in folder
files2 = []
files2 = [pd.read_csv(file, delimiter='\t') for file in glob.glob(os.path.join(files_SI_folder, "*.txt"))]

# Concatenate dataframe list
df_justice = pd.concat(files1)
df_injustice = pd.concat(files2)

#drop nans from rows with nan in column 'DE'

df_justice = df_justice.dropna(subset=['DE'])
df_injustice = df_injustice.dropna(subset=['DE'])

#reset index of DataFrame
df_justice = df_justice.reset_index(drop=True)
df_injustice = df_injustice.reset_index(drop=True)

# extract just the keywords from each
"""
This list of keywords is not tokenized yet
"""

justice_keywords = df_justice['DE'].tolist()
injustice_keywords = df_injustice['DE'].tolist()

# see the length of both just to confirm you got everything
print('social justice data is: ' + str(len(df_justice)))
print('social injustice data is: ' + str(len(df_injustice)))


social justice data is: 6736
social injustice data is: 359


# Step 2
## create a function to take list of list, flatten and clean
This tokenizes the list of keywords but maintains phrases, strips out punctuation, and lowers

the function below cleans a list of lists

In [3]:
def thecleanerator(some_list):
    sneeps = []
    for i in range(len(some_list)):

        peeps = some_list[i].lower().split(';')
        fleeps = [s for s in peeps if not re.search(r'\d',s)]
        bleeps = []
        for i in range(len(fleeps)):
            a = fleeps[i].translate(str.maketrans('', '', string.punctuation)).lstrip()
            bleeps.append(a)

        sneeps.append(bleeps)
    return(sneeps)


In [4]:
### clean and flatten the justice list
justice_list = thecleanerator(justice_keywords)
justice_list = [num for sublist in justice_list for num in sublist]
#clean and flatten the injustice list
injustice_list = thecleanerator(injustice_keywords)
injustice_list = [num for sublist in injustice_list for num in sublist]

print('the justice list is this long: ' + str(len(justice_list)))
print('*************')

print('the injustice list is this long: ' + str(len(injustice_list)))

the justice list is this long: 36809
*************
the injustice list is this long: 1986


# Step 3: Count terms in each list
One list will be called n_sj and the other n_si. <br>
These will be used to calculate % of terms from search 1 or search 2, and for weighting,
(which is the normalized number of times a term appears in each respective search, NOT the entire corpus)

In [15]:
# create df for our lists
df_si = pd.DataFrame(injustice_list, columns = ['terms_si'])
df_sj = pd.DataFrame(justice_list, columns = ['terms_sj'])

# term counts for each search list - counts have been normalized
counts_of_SJ = df_sj['terms_sj'].value_counts(ascending = False, 
                                              normalize = True, dropna=True)
counts_of_si = df_si['terms_si'].value_counts(ascending = False, 
                                              normalize = True, dropna=True)
"""
May consider adding a truncation here to limit both sets of normalized counts to 500 each.
This would sum to a 1000 and limit one set dominating the other when truncated down below.
change 02242022
"""
#truncate both sets to only 500 of the top value counts
trunc_counts_of_SJ = counts_of_SJ[:500]
trunc_counts_of_si = counts_of_si[:500]

#concat both lists together into new df...
df_results = pd.concat([trunc_counts_of_SJ, trunc_counts_of_si], axis=1).fillna(0)

"""
end of change 02242022
"""

#convert index to column creating a label column 
df_results.reset_index(inplace=True)
#rename column
df_results.rename(columns = {'index':'label'}, inplace = True)

# create a total terms count column
df_results['weight'] = df_results.terms_sj + df_results.terms_si

#create percentages
df_results['P_si_in_sj'] = df_results.terms_si / df_results.weight
df_results['P_SJ_in_si'] = df_results.terms_sj / df_results.weight

#invert one column to enable use later in visualization to seperate one search
# from another. Search 1 will be positive, and search 2 is negaitive, for example
# social injustice is -1 and social justice remains +1
df_results['P_si_in_sj'] = df_results['P_si_in_sj'].mul(-1)

# create indicator column
df_results['indicator'] = df_results.P_si_in_sj + df_results.P_SJ_in_si

#drop first index because its showing nan for some weird reason....
#df_results.drop(df_results.index[0], inplace = True)

#see the df
df_results.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   label       851 non-null    object 
 1   terms_sj    851 non-null    float64
 2   terms_si    851 non-null    float64
 3   weight      851 non-null    float64
 4   P_si_in_sj  851 non-null    float64
 5   P_SJ_in_si  851 non-null    float64
 6   indicator   851 non-null    float64
dtypes: float64(6), object(1)
memory usage: 46.7+ KB


# Step 4: change to match what gephi needs for import
create two dataframes to export, one for nodes, one for edges
- [ ] create unique id for nodes and other columns for node attributes
    - [x] id (unique)
    - [x] label (term)
    - [x] indicator (node color) 
    - [x] weight (node size)
    - [x] interval (year)

- [x] create matrix of co-occurence and export this to Gephi 
    - [ ] can also be used later for Networkx
    - [x] add year for interval

In [16]:
# create new dataframe for nodes
df_nodes = df_results[['label','weight','indicator']].copy()


#use the index as an id number
df_nodes.reset_index(inplace=True)
df_nodes = df_nodes.rename(columns = {'index':'id'}) # now, each term has a unique id number, but there should still be duplicates

"""
sort by weight which is the term counts normalized and should preserve ones of each that are most frequent
regardless of which search it is, and how frequently one may occur in the other search
"""

df_nodes.sort_values('weight', 
                     ascending = False, 
                     inplace = True,
                    ignore_index = True
                    )
#sorted by weight
df_nodes

,id,label,weight,indicator
0,0,social justice,0.099271,0.391331
1,500,social injustice,0.014099,-1.000000
2,3,neoliberalism,0.009599,-0.049081
3,8,justice,0.008552,-0.059745
4,1,equity,0.008237,0.266468
...,...,...,...,...
846,496,postcolonial theory,0.000217,1.000000
847,497,program evaluation,0.000217,1.000000
848,498,reflective practice,0.000217,1.000000
849,499,iran,0.000217,1.000000


# Step 5: Edges
Use nodes file to make an edges list from the same nodes. 

In [18]:
#reference: https://pythondata.com/text-analytics-visualization/

"""
Truncation happended above on 02242022
delete the commented line out?
"""
#edges_base_df = df_nodes.truncate(after = 2000)
edges_base_df = df_nodes
edges_base_df

,id,label,weight,indicator
0,0,social justice,0.099271,0.391331
1,500,social injustice,0.014099,-1.000000
2,3,neoliberalism,0.009599,-0.049081
3,8,justice,0.008552,-0.059745
4,1,equity,0.008237,0.266468
...,...,...,...,...
846,496,postcolonial theory,0.000217,1.000000
847,497,program evaluation,0.000217,1.000000
848,498,reflective practice,0.000217,1.000000
849,499,iran,0.000217,1.000000


In [19]:
edges_label_list = edges_base_df['label'].to_list()
edges_id_list = edges_base_df['id'].to_list()

In [20]:
#concat both into one corpus
corpus_list = [df_justice, df_injustice]
df_corpus = pd.concat(corpus_list)

#create a big ol' list of the keywords from all articles in both searches
corpus_keywords = df_corpus['DE'].to_list()# create a list of strings


#### cleaning function
needed a cleaning function different than the one above. <br>
mebe the one above should be reduced to this? 

In [21]:
clean_corpus_keywords = thecleanerator(corpus_keywords)

#if you want to see
clean_corpus_keywords[1][1]

'empowerment'

In [22]:
"""
so, now we have two lists:
clean_corpus_keywords which is a list of list of cleaned terms to search
and
edges_label_list which is a list of the top 1000 most frequent terms from the 
df_nodes dataframe which was already exported
"""
document = clean_corpus_keywords
edges_df = pd.DataFrame()

# Step 6: Co-occurrence edge list
## WARNING! This can take a long time!
#### references:
for the following cell
https://cmsdk.com/python/cooccurrence-matrix-from-list-of-words-in-python.html

In [23]:
import math
from tqdm import tqdm
lst_dict = []
for a in tqdm(edges_label_list):
    for b in edges_label_list:
        count = 0
        for x in document:
            if a != b:
                if a in x and b in x:
                    count += 1
            else:
                n = x.count(a)
                if n >= 2:
                    count += math.factorial(n)/math.factorial(n - 2)/2
        if count != 0:
            lst_dict.append({'source':a, 'target':b, 'weight':count})

100%|██████████| 851/851 [11:33<00:00,  1.23it/s]


### Oh my crickets! an edges list!

In [24]:
cols = ['source', 'target', 'weight']
df1 = pd.DataFrame(columns=cols)
edges_df = df1.append(lst_dict)
edges_df

,source,target,weight
0,social justice,neoliberalism,58.0
1,social justice,justice,16.0
2,social justice,equity,110.0
3,social justice,gender,41.0
4,social justice,human rights,92.0
...,...,...,...
23004,degrowth,political ecology,1.0
23005,degrowth,urban planning,1.0
23006,degrowth,public space,1.0
23007,degrowth,economy,1.0


### save the time consuming data as a pickle
just in case you need to restart the kernal at any point. 

In [25]:
### exports the dataframe to perform a leiden algorithm for clusters in the 
### Leiden Algorithm notebook
# can be deleted once everything is hunky dorey

#edges_df.to_csv("edges_df.csv", index=False)

#OR
import pickle

pickle.dump(edges_df, open('edges_df.pickle', 'wb'))

In [26]:
# to read data back in if needed. 
data = pickle.load(open('edges_df.pickle', 'rb'))

# Leiden Algorithm for clustering

source: https://leidenalg.readthedocs.io/en/stable/intro.html

and GitHub: https://github.com/vtraag/leidenalg

leidenalg.find_partition(graph, partition_type, initial_membership=None, weights=None, n_iterations=2, max_comm_size=0, seed=None, **kwargs)¶
#### see: https://leidenalg.readthedocs.io/en/stable/reference.html

## for igraph:
https://igraph.org/python/

### if you have trouble installing:
https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
or try installing from a terminal 


# Step 7: import Leiden Algorithm and dependencies

In [30]:
pip install leidenalg
# you may need to install cairocffi as a dependency
### if the notebook is giving a syntax error, try installing from a console in the notebook

SyntaxError: invalid syntax (315943597.py, line 1)

In [31]:
import igraph as ig
import leidenalg as la
import cairocffi

# Step 8: run the Leiden algorithm

In [32]:
#rename edges_df to df for simplicity
df = edges_df # this is the 1000 top weighted keywords

# creates a graph from the dataframe using tuples by index
G = ig.Graph.TupleList(df.itertuples(index=False), directed=False, weights=True)

# there are other partition options here that may be worth explorting at some time...
partition = la.find_partition(G, la.ModularityVertexPartition)

# adds cluster info to G
G.vs['cluster'] = partition.membership

# IF you want to see a messy network map
# not neccessary for processing data
#ig.plot(partition,vertex_size = 30)

# Step 9: export a graph file for Gephi
After running the cell below, go into Gephi and perform the following steps:

- [ ] Import the graph file into Gephi. 
- [ ] Export the nodes sheet back out of Gephi. 
- [ ] Return here to Step 11

Why? Well, I don't understand what G is as a datatype. I know that I can export it as 
a graph file, but exporting it as an edges file does not include the cluster data. 
So, this is a messy, but effective way of dealing with the problem, for now. 

In [33]:
# exports a graph file to open in Gephi
ig.save(G,'edges_graph.gml')

# Step 10: import nodes with clusters
Now to reassign cluster to each node, so that nodes has id, label, weight, indicator and cluster. 

In [34]:
import pandas as pd
#df = pd.read_csv("nodes_w_clusters.csv", header=None, delimiter=r"\s+")
df_nodes_w_clusters = pd.read_csv("nodes_w_clusters.csv")
#df.rename(columns = {0:'source', 1:'target', 3:'label', 4:'cluster'}, inplace = True)
df_nodes_w_clusters



,Id,Label,timeset,name,cluster
0,0,0,NaN,social justice,4
1,1,1,NaN,neoliberalism,2
2,2,2,NaN,justice,0
3,3,3,NaN,equity,3
4,4,4,NaN,gender,1
...,...,...,...,...,...
846,846,846,NaN,rivaz alice,15
847,847,847,NaN,cinema and education,16
848,848,848,NaN,paradise travel,16
849,849,849,NaN,auto learning experience,16


In [35]:
print(edges_df.info())
print(df_nodes.info())
print(df_nodes_w_clusters.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23009 entries, 0 to 23008
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   source  23009 non-null  object 
 1   target  23009 non-null  object 
 2   weight  23009 non-null  float64
dtypes: float64(1), object(2)
memory usage: 719.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         851 non-null    int64  
 1   label      851 non-null    object 
 2   weight     851 non-null    float64
 3   indicator  851 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 26.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851 entries, 0 to 850
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Id       851 non-null    int64  
 1   Label    851 

## create an xlookup function

In [36]:

# create xlookup function to apply across dataframes
# code from: https://pythoninoffice.com/replicate-excel-vlookup-hlookup-xlookup-in-python/

def xlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'0' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

# Step 12: apply xlookup function
1. First to replace the keywords in source and target in the edges df wtih their IDs.Othewise, gephi assigns ID's and its a mess. 

2. Then, apply xlookup function to the nodes file to add cluster to the keywords. 
3. Then filter or query for nodes having a cluster. 


In [37]:
"""
the following replaces the terms in source and
target with their respective id numbers from 
the nodes dataframe
"""

#make copy of the dataframe
edges_df2 = edges_df.copy(deep=True)

#replace source with its id
edges_df2['source2'] = edges_df['source'].apply(xlookup, args = (
    df_nodes['label'],
    df_nodes['id']
))

#replace target with its id
edges_df2['target2'] = edges_df['target'].apply(xlookup, args = (
    df_nodes['label'],
    df_nodes['id']
))

# drop the original ones
edges_df2.drop(['source'], axis = 1, inplace = True)
edges_df2.drop(['target'], axis = 1, inplace = True)

#rename columns
edges_df2.rename(columns = {'source2':'source',
                           'target2':'target'
                           }, inplace = True)

#lastly, reindex and reorder the columns
edges_df2.reindex(columns=['source', 'target', 'weight'])


,source,target,weight
0,0,3,58.0
1,0,8,16.0
2,0,1,110.0
3,0,5,41.0
4,0,6,92.0
...,...,...,...
23004,490,250,1.0
23005,490,340,1.0
23006,490,347,1.0
23007,490,429,1.0


# Step 11: export the edges file

In [38]:
edges_df2.to_csv(r'edges.csv', index = False)

# Step 12: add cluster info to the nodes list

In [40]:
#make copy of the dataframe
nodes_df2 = df_nodes

nodes_df2

,id,label,weight,indicator
0,0,social justice,0.099271,0.391331
1,500,social injustice,0.014099,-1.000000
2,3,neoliberalism,0.009599,-0.049081
3,8,justice,0.008552,-0.059745
4,1,equity,0.008237,0.266468
...,...,...,...,...
846,496,postcolonial theory,0.000217,1.000000
847,497,program evaluation,0.000217,1.000000
848,498,reflective practice,0.000217,1.000000
849,499,iran,0.000217,1.000000


In [41]:
df_nodes_w_clusters

,Id,Label,timeset,name,cluster
0,0,0,NaN,social justice,4
1,1,1,NaN,neoliberalism,2
2,2,2,NaN,justice,0
3,3,3,NaN,equity,3
4,4,4,NaN,gender,1
...,...,...,...,...,...
846,846,846,NaN,rivaz alice,15
847,847,847,NaN,cinema and education,16
848,848,848,NaN,paradise travel,16
849,849,849,NaN,auto learning experience,16


In [42]:
#replace source with its id
# destination = lookup value, source array, replace array
nodes_df2['clusters'] = nodes_df2['label'].apply(xlookup, args = (
    df_nodes_w_clusters['name'],
    df_nodes_w_clusters['cluster']
))

In [43]:
nodes_df2

,id,label,weight,indicator,clusters
0,0,social justice,0.099271,0.391331,4
1,500,social injustice,0.014099,-1.000000,6
2,3,neoliberalism,0.009599,-0.049081,2
3,8,justice,0.008552,-0.059745,0
4,1,equity,0.008237,0.266468,3
...,...,...,...,...,...
846,496,postcolonial theory,0.000217,1.000000,1
847,497,program evaluation,0.000217,1.000000,0
848,498,reflective practice,0.000217,1.000000,3
849,499,iran,0.000217,1.000000,0


# Step 13: export nodes to csv
Has been truncated to top 1000 by weight (occ).

In [44]:
nodes_df2.to_csv(r'nodes.csv', index = False)


# That's it! 
You should now have data in Gephy to compare two search results. 

## important notes:
### Truncation is occuring based on normalized weight (proportional frequency counts.) 
This was based on the thinking that one would want to see the most frequently used terms. Truncation could occur based on indicator (percentage of one term found in the other,) but this truncation would need to be based on absolute value and of course, you'd only get the extremes represented and not shared words. 
### import data
imports are Full Record with Cited Refernces from WoS exported in 500 chunks as tab-seperated values text files, (.txt)

### Frequency is based on keyword/keyphrases only. 
This does not look at frequency of keywords appearing in the article (abstract or full text.) <br>
It also does not look at the place in the text, (which is its own technique)



